LSTM FOR FORECASTING TB CASES AT KIDH

INSTALL NECESSARY LIBRARY

In [ ]:
#pip install --ignore-installed qiskit-terra qiskit-aer

In [ ]:
#pip install numpy==1.19.5

In [ ]:
pip install -U scikit-learn

DATASET UPLOAD TO COLAB

IMPORT NECCESSARY LIBRARY

In [ ]:
#upload the dataset into google colab
from google.colab import files
uploaded = files.upload()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from sklearn.preprocessing import MinMaxScaler
from keras.layers.core import Dense, Activation, Dropout
import time #helper libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(9,6), 'figure.dpi':100})
plt.style.use('fivethirtyeight')
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['text.color'] = 'G'

READ THE DATASET

In [ ]:
df = pd.read_csv('residual_new.csv')

In [ ]:
df

DATA PREPROSSING

In [ ]:
df['Date']= pd.to_datetime(df['date'],format="%d/%m/%y")
df.drop(columns=['date'], inplace=True)

In [ ]:
series=df.set_index('Date')

In [ ]:
series.plot(figsize = (19, 8),title="Monthly New and Relapse (bacteriologically confirmed or clinical diagnosis) TB incidence at Kibongoto Infectious Disease Hospital  from 2015 to 2020.",ylabel="New and Relapse TB cases",xlabel="Year")
plt.show()

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)
# load the dataset
dataset = series.values
dataset = dataset.astype('float32')

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)




In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(-1, 1))
dataset = scaler.fit_transform(dataset)
# split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
# reshape into X=t and Y=t+1
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
testX.shape

CREATE MODEL AND FITS

In [ ]:
batch_size = 1
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True,return_sequences=True))
#model.add(Dropout(0.2))
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
#model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
for i in range(500):
	model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()

In [ ]:
# make forecast
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)


In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

EVALUATE A MODEL

TRAINING PERFOMANCE METRIC

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = mean_absolute_percentage_error(actual, forecast)  # MAPE
    mse=mean_squared_error(actual, forecast, squared=False)
    mae = mean_absolute_error(actual, forecast)    # MAE   # MPE
    rmse = np.sqrt(mean_squared_error(actual, forecast))
    r2=r2_score(actual, forecast)  # RMSE
    return({'mape':mape,  'mae': mae, "mse":mse,
             'rmse':rmse,'r2_score':r2})

forecast_accuracy(trainPredict[:,0],trainY[0])

TESTING PERFOMANCE METRIC

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = mean_absolute_percentage_error(actual, forecast)  # MAPE
    mse=mean_squared_error(actual, forecast, squared=False)
    mae = mean_absolute_error(actual, forecast)    # MAE   # MPE
    rmse = np.sqrt(mean_squared_error(actual, forecast))
    r2=r2_score(actual, forecast)  # RMSE
    return({'mape':mape,  'mae': mae, "mse":mse,
             'rmse':rmse,'r2_score':r2})

forecast_accuracy(testPredict[:,0],testY[0])

TESTING

In [ ]:
import pandas as pd

import numpy as np

%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score

from keras.models import Sequential

from keras.layers import Dense

from keras.callbacks import EarlyStopping

from keras.optimizers import Adam

from keras.layers import LSTM

In [ ]:
series

In [ ]:
split_date = pd.Timestamp('2019-01-01')

df =  series['tbcases']

train = df.loc[:split_date]

test = df.loc[split_date:]

plt.figure(figsize=(10, 6))

ax = train.plot()

test.plot(ax=ax)

plt.legend(['train', 'test']);

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train=train.values
train=train.reshape(-1,1)
train_sc = scaler.fit_transform(train)
test=test.values
test=test.reshape(-1,1)
test_sc = scaler.transform(test)



In [ ]:
X_train = train_sc[:-1]

y_train = train_sc[1:]



X_test = test_sc[:-1]

y_test = test_sc[1:]

In [ ]:
nn_model = Sequential()

nn_model.add(Dense(512, input_dim=1, activation='tanh'))
nn_model.add(Dense(512, input_dim=1, activation='tanh'))
nn_model.add(Dense(1))

nn_model.compile(loss='mean_squared_error', optimizer='adam')

early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)

history = nn_model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop], shuffle=False)

In [ ]:
y_pred_test_nn = nn_model.predict(X_test)

y_train_pred_nn = nn_model.predict(X_train)

print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_nn)))

print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_nn)))

In [ ]:
X_train_lmse = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_lmse = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

lstm_model = Sequential()
lstm_model.add(LSTM(100, input_shape=(1, X_train_lmse.shape[1]), activation='tanh', kernel_initializer='lecun_uniform', return_sequences=False))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
history_lstm_model = lstm_model.fit(X_train_lmse, y_train, epochs=100, batch_size=1, verbose=1, shuffle=False, callbacks=[early_stop])


In [ ]:
y_pred_test_lstm = lstm_model.predict(X_test_lmse)
y_train_pred_lstm = lstm_model.predict(X_train_lmse)
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_lstm)))
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_lstm)))
